In [4]:
import sys
sys.path.append("..")

In [10]:
import pandas as pd
import numpy as np
from src.model_fit import fit_model, plot_model

bulk_df = pd.read_parquet('../data_interim/bulk_df.parquet')
binned_df = pd.read_parquet('../data_interim/binned2H0.parquet')

In [12]:
binned_df

,H6_mean,H6_std,H6_count,L0_mean,L0_std,L0_count,n_mean,n_std,n_count,H6_sem,L0_sem,n_sem
0,0.770708,0.089281,299,118.964837,34.346587,299,-0.085419,0.016798,299,0.005163,1.986314,0.000971
1,1.117240,0.097499,830,87.150302,25.219482,830,-0.071381,0.022625,830,0.003384,0.875381,0.000785
2,1.424999,0.094691,855,89.864894,33.032562,855,-0.050971,0.032642,855,0.003238,1.129690,0.001116
3,1.795739,0.099235,412,133.274394,41.535792,412,0.009150,0.062200,412,0.004889,2.046322,0.003064
4,2.139357,0.102592,313,129.620490,42.298495,313,0.060974,0.068613,313,0.005799,2.390852,0.003878
5,2.486148,0.095803,247,130.864470,52.963961,247,0.106557,0.074637,247,0.006096,3.370016,0.004749
6,2.808494,0.091340,119,148.535520,63.538552,119,0.140386,0.094629,119,0.008373,5.824570,0.008675
7,3.194990,0.107594,85,162.909295,53.619279,85,0.192049,0.138750,85,0.011670,5.815827,0.015050
8,3.533343,0.097582,43,163.741495,56.370292,43,0.236115,0.125967,43,0.014881,8.596389,0.019210
9,3.909239,0.095549,23,134.133526,53.348505,23,0.288935,0.178762,23,0.019923,11.123932,0.037274


trying out this model_fit script, which is taken from the cells written on 12-17

In [11]:
# 1. Define Model
def monismith_offset(X, a, b, c, d):
    H, L = X
    return a * (H**b) * (L**c) + d

# 2. Run Fit
res_mon = fit_model(
    model_func=monismith_offset,
    binned_df=binned_df,
    x_cols=['H0_mean', 'L0_mean'],  # Columns in binned_df
    p0=[0.1, -2.0, 0.0, -0.15],
    bounds=([0, -5, -5, -0.5], [np.inf, 5, 5, 0.5]),
    label="Monismith Fit"
)

# 3. Plot
# You can plot instantly:
plot_model(res_mon, binned_df, bulk_df, raw_x_cols=['H6', 'L0']) 
plt.show()

# OR make subplots like before:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
plot_model(res_mon, binned_df, bulk_df, raw_x_cols=['H6', 'L0'], ax=axes[0])
# plot_model(res_stockdon, ..., ax=axes[1])
plt.show()

/Users/isidorarojas/Desktop/DIRECTORY/2025/reef_runup/moku_PILOT/Mark_Meetings/../src/model_fit.py:93: SyntaxWarning: invalid escape sequence '\c'
  f"RMSE={fit_result['rmse']:.3f} | $\chi_\\nu^2$={fit_result['chi_sq']:.2f}")


KeyError: "['H0_mean'] not in index"